## 3-phase Composite Example

In [1]:
from hashin_shtrikman_mp.core.user_input import MaterialProperty, Material, MixtureProperty, Mixture, UserInput
from hashin_shtrikman_mp.core import GeneticAlgorithm
from hashin_shtrikman_mp.core.genetic_algorithm import OptimizationParams
from hashin_shtrikman_mp.core.visualization import OptimizationResultVisualizer
from hashin_shtrikman_mp.core.match_finder import MatchFinder

Define properties for each material

In [2]:
properties_mat_1 = [
    MaterialProperty(prop='elec_cond_300k_low_doping',  upper_bound=120, lower_bound=1e-7),
    MaterialProperty(prop='therm_cond_300k_low_doping', upper_bound=2,   lower_bound=1e-7),
    MaterialProperty(prop='bulk_modulus',               upper_bound=500, lower_bound=50),
    MaterialProperty(prop='shear_modulus',              upper_bound=500, lower_bound=80),
    MaterialProperty(prop='universal_anisotropy',       upper_bound=6,   lower_bound=1),
]

properties_mat_2 = [
    MaterialProperty(prop='elec_cond_300k_low_doping',  upper_bound=78,  lower_bound=1e-7),
    MaterialProperty(prop='therm_cond_300k_low_doping', upper_bound=2,   lower_bound=1e-7),
    MaterialProperty(prop='bulk_modulus',               upper_bound=400, lower_bound=20),
    MaterialProperty(prop='shear_modulus',              upper_bound=500, lower_bound=100),
    MaterialProperty(prop='universal_anisotropy',       upper_bound=4.3, lower_bound=1.3),
]

properties_mat_3 = [
    MaterialProperty(prop='elec_cond_300k_low_doping',  upper_bound=78,  lower_bound=1e-7),
    MaterialProperty(prop='therm_cond_300k_low_doping', upper_bound=2,   lower_bound=1e-7),
    MaterialProperty(prop='bulk_modulus',               upper_bound=700, lower_bound=20),
    MaterialProperty(prop='shear_modulus',              upper_bound=600, lower_bound=100),
    MaterialProperty(prop='universal_anisotropy',       upper_bound=2.1, lower_bound=0.9),
]

Create an instance of the `Material` class for each material

In [3]:
mat_1   = Material(name='mat_1', properties=properties_mat_1)
mat_2   = Material(name='mat_2', properties=properties_mat_2)
mat_3   = Material(name='mat_3', properties=properties_mat_3)

Define properties for the mixture

In [4]:
properties_mixture = [
    MixtureProperty(prop='elec_cond_300k_low_doping',  desired_prop=9),
    MixtureProperty(prop='therm_cond_300k_low_doping', desired_prop=0.9),
    MixtureProperty(prop='bulk_modulus',               desired_prop=280),
    MixtureProperty(prop='shear_modulus',              desired_prop=230),
    MixtureProperty(prop='universal_anisotropy',       desired_prop=1.5),
]

Create an instance of the `Mixture` class to store the desired composite properties and aggregate the materials and mixture into a list for later iteration

In [5]:
mixture = Mixture(name='mixture', properties=properties_mixture)
aggregate = [mat_1, mat_2, mat_3, mixture]

Create an instance of the `UserInput` class from the materials and mixture(s) just created

In [6]:
user_input= UserInput(materials=[mat_1, mat_2, mat_3], mixtures=[mixture])

Iterate over the aggregated materials and mixture to populate a dictionary which stores the upper and lower search bounds for every material property of interest

In [7]:
# Initialize dictionaries to store the overall upper and lower bounds for each property
bounds_dict = {}

# Iterate over materials
for entity in aggregate:

    # Skip the mixture as it doesn't have upper and lower bounds
    if isinstance(entity, Material):
        
        for property in entity.properties:
            prop_name = property.prop

            # Add a key for the property if bounds for that property are not already present
            if prop_name not in bounds_dict:
                bounds_dict[prop_name] = {
                    'upper_bound': property.upper_bound,
                    'lower_bound': property.lower_bound
                    }
            else:
                # Update overall upper and lower bounds by comparing with existing values
                bounds_dict[prop_name]['upper_bound'] = max(bounds_dict[prop_name]['upper_bound'], property.upper_bound)
                bounds_dict[prop_name]['lower_bound'] = min(bounds_dict[prop_name]['lower_bound'], property.lower_bound)

Print the overall bounds for each property (bounds across all materials)

In [8]:
for prop, bounds in bounds_dict.items():
    print(f"Property: {prop}\n\tUpper Bound: {bounds['upper_bound']}\n\tLower Bound: {bounds['lower_bound']}\n")

Property: elec_cond_300k_low_doping
	Upper Bound: 120.0
	Lower Bound: 1e-07

Property: therm_cond_300k_low_doping
	Upper Bound: 2.0
	Lower Bound: 1e-07

Property: bulk_modulus
	Upper Bound: 700.0
	Lower Bound: 20.0

Property: shear_modulus
	Upper Bound: 600.0
	Lower Bound: 80.0

Property: universal_anisotropy
	Upper Bound: 6.0
	Lower Bound: 0.9



Initialize an instance of the `OptimizationParams` class using the previously defined `UserInput`, as well as an instance of the `GeneticAlgorithm` class

In [9]:
optimization_parameters = OptimizationParams.from_user_input(user_input)
ga = GeneticAlgorithm()

2025-03-30 13:41:32,371 - hashin_shtrikman_mp.log.custom_logger - INFO - Loading property categories from /Users/carlabecker/Library/Mobile Documents/com~apple~CloudDocs/Carla's Desktop/UC Berkeley/Research/Materials Project/hashin_shtrikman_mp/src/hashin_shtrikman_mp/io/inputs/data/mp_property_docs.yaml.
2025-03-30 13:41:32,379 - hashin_shtrikman_mp.log.custom_logger - INFO - property_categories = ['carrier-transport', 'elastic']
2025-03-30 13:41:32,386 - hashin_shtrikman_mp.log.custom_logger - INFO - mixture_props = {'elec_cond_300k_low_doping': {'desired_prop': 9.0}, 'therm_cond_300k_low_doping': {'desired_prop': 0.9}, 'bulk_modulus': {'desired_prop': 280.0}, 'shear_modulus': {'desired_prop': 230.0}, 'universal_anisotropy': {'desired_prop': 1.5}}


Check that `property_docs` from `src/hashin_shtrikman_mp/io/inputs/data/mp_property_docs.yaml` has been loaded correctly into the instance of `OptimizationParams`.

For reading bulk and shear moduli from the Materials Project database, there are a few options outlined [here](https://docs.materialsproject.org/methodology/materials-methodology/elasticity). In `hashin_shtrikman_mp` we use the Voigt-Reuss-Hill average.

In [10]:
for cat in optimization_parameters.property_docs.keys():
    print(f'Property category: {cat}')
    for prop in optimization_parameters.property_docs[cat].keys():
        option = optimization_parameters.property_docs[cat][prop]
        if option == None:
            print(f"\tProperty: {prop}")
        else:
            print(f"\tProperty: {prop}, Averaging type: {option}")

Property category: carrier-transport
	Property: elec_cond_300k_low_doping
	Property: therm_cond_300k_low_doping
Property category: dielectric
	Property: e_electronic
	Property: e_ionic
	Property: e_total
	Property: n
Property category: elastic
	Property: bulk_modulus, Averaging type: vrh
	Property: shear_modulus, Averaging type: vrh
	Property: universal_anisotropy
Property category: magnetic
	Property: total_magnetization
	Property: total_magnetization_normalized_vol
Property category: piezoelectric
	Property: e_ij_max


Check that the number of materials in the composite and the number of properties of interest have been loaded correctly from `UserInput`

In [11]:
print('Number of Materials:', optimization_parameters.num_materials)
print('Number of Properties:', optimization_parameters.num_properties)

Number of Materials: 3
Number of Properties: 6


Check that lower and upper bounds for material property searches have been loaded correctly from `UserInput`

In [12]:
# Check lower bounds
print('\nUser-defined lower bounds: ')
for mat in optimization_parameters.lower_bounds.keys():
    if type(optimization_parameters.lower_bounds[mat]) is dict:
        print(f'  {mat}:')
        for cat in optimization_parameters.lower_bounds[mat].keys():
            print(f'    {cat}:')
            props = list(optimization_parameters.property_docs[cat].keys())
            lower_bounds = optimization_parameters.lower_bounds[mat][cat]
            for prop, lower_bound in zip(props, lower_bounds):
                print(f'      {prop}: {lower_bound}')

# Check upper bounds
print('\nUser-defined upper bounds: ')
for mat in optimization_parameters.upper_bounds.keys():
    if type(optimization_parameters.upper_bounds[mat]) is dict:
        print(f'  {mat}:')
        for cat in optimization_parameters.upper_bounds[mat].keys():
            print(f'    {cat}:')
            props = list(optimization_parameters.property_docs[cat].keys())
            upper_bounds = optimization_parameters.upper_bounds[mat][cat]
            for prop, upper_bound in zip(props, upper_bounds):
                print(f'      {prop}: {upper_bound}')


User-defined lower bounds: 
  mat_1:
    carrier-transport:
      elec_cond_300k_low_doping: 1e-07
      therm_cond_300k_low_doping: 1e-07
    elastic:
      bulk_modulus: 50.0
      shear_modulus: 80.0
      universal_anisotropy: 1.0
  mat_2:
    carrier-transport:
      elec_cond_300k_low_doping: 1e-07
      therm_cond_300k_low_doping: 1e-07
    elastic:
      bulk_modulus: 20.0
      shear_modulus: 100.0
      universal_anisotropy: 1.3
  mat_3:
    carrier-transport:
      elec_cond_300k_low_doping: 1e-07
      therm_cond_300k_low_doping: 1e-07
    elastic:
      bulk_modulus: 20.0
      shear_modulus: 100.0
      universal_anisotropy: 0.9

User-defined upper bounds: 
  mat_1:
    carrier-transport:
      elec_cond_300k_low_doping: 120.0
      therm_cond_300k_low_doping: 2.0
    elastic:
      bulk_modulus: 500.0
      shear_modulus: 500.0
      universal_anisotropy: 6.0
  mat_2:
    carrier-transport:
      elec_cond_300k_low_doping: 78.0
      therm_cond_300k_low_doping: 2.0
    

Check that the desired properties have been loaded correctly from `UserInput`

In [13]:
print('User-defined desired properties:', )

for cat in optimization_parameters.desired_props.keys():
    print(f'  {cat}:')
    props = list(optimization_parameters.property_docs[cat].keys())
    desired_props = optimization_parameters.desired_props[cat]
    for prop, desired_prop in zip(props, desired_props):
        print(f'    {prop}: {desired_prop}')

User-defined desired properties:
  carrier-transport:
    elec_cond_300k_low_doping: 9.0
    therm_cond_300k_low_doping: 0.9
  elastic:
    bulk_modulus: 280.0
    shear_modulus: 230.0
    universal_anisotropy: 1.5


Run the optimization $n=3$ materials. Identify the optimal properties of each material and the respective volume fractions that achieve the desired composite material properties.

In [14]:
ga_result = ga.run(user_input, gen_counter=True)

2025-03-30 13:41:32,608 - hashin_shtrikman_mp.log.custom_logger - INFO - Loading property categories from /Users/carlabecker/Library/Mobile Documents/com~apple~CloudDocs/Carla's Desktop/UC Berkeley/Research/Materials Project/hashin_shtrikman_mp/src/hashin_shtrikman_mp/io/inputs/data/mp_property_docs.yaml.
2025-03-30 13:41:32,616 - hashin_shtrikman_mp.log.custom_logger - INFO - property_categories = ['carrier-transport', 'elastic']
2025-03-30 13:41:32,617 - hashin_shtrikman_mp.log.custom_logger - INFO - mixture_props = {'elec_cond_300k_low_doping': {'desired_prop': 9.0}, 'therm_cond_300k_low_doping': {'desired_prop': 0.9}, 'bulk_modulus': {'desired_prop': 280.0}, 'shear_modulus': {'desired_prop': 230.0}, 'universal_anisotropy': {'desired_prop': 1.5}}


Generation 0 of 100
Generation 1 of 100
Generation 2 of 100
Generation 3 of 100
Generation 4 of 100
Generation 5 of 100
Generation 6 of 100
Generation 7 of 100
Generation 8 of 100
Generation 9 of 100
Generation 10 of 100
Generation 11 of 100
Generation 12 of 100
Generation 13 of 100
Generation 14 of 100
Generation 15 of 100
Generation 16 of 100
Generation 17 of 100
Generation 18 of 100
Generation 19 of 100
Generation 20 of 100
Generation 21 of 100
Generation 22 of 100
Generation 23 of 100
Generation 24 of 100
Generation 25 of 100
Generation 26 of 100
Generation 27 of 100
Generation 28 of 100
Generation 29 of 100
Generation 30 of 100
Generation 31 of 100
Generation 32 of 100
Generation 33 of 100
Generation 34 of 100
Generation 35 of 100
Generation 36 of 100
Generation 37 of 100
Generation 38 of 100
Generation 39 of 100
Generation 40 of 100
Generation 41 of 100
Generation 42 of 100
Generation 43 of 100
Generation 44 of 100
Generation 45 of 100
Generation 46 of 100
Generation 47 of 100
Ge

Create an instance of the `OptimizationResultVisualizer` class using the genetic algorithm result

In [15]:
visualizer = OptimizationResultVisualizer(ga_result)

Print the optimization results as a table. Each row represents a *theoretical* material.

In [16]:
visualizer.print_table_of_best_designs(rows=10)

Plot the genetic algorithm convergence plot

In [17]:
visualizer.plot_optimization_results()

Plot the contributions to the cost function for the best performer

*Note on how many contributions to expect:*
Per property, there is
* 1 effective property term
* (Non-modulus) 2 concentration factor terms per material
* (Modulus) 2 concentration factor terms per coupled (bulk, shear) moduli per material -- not considered individual properties

*For example*, for a 3-phase composite which considers carrier-transport and elastic property categories, not including volume fraction, there are 5 properties per material, so in total we expect
* 5 effective property terms
* 18 = 6 concentration factor terms (load and response terms for electrical conductivity, thermal conductivity, and universal anisotropy) $\times$ 3 materials
* 6 = 2 concentration factor terms (coupled deviatoric and hydrostatic terms from coupled bulk/shear moduli) $\times$ 3 materials

In [18]:
visualizer.plot_cost_func_contribs()

Cost: 0.04868656090766206, Number effective properties: 5, Number of concentration factors: 24


Create an instance of the `MatchFinder` class using the genetic algorithm result

In [19]:
match_finder = MatchFinder(ga_result)

Finally, get *real* material matches

In [20]:
matches_dict = match_finder.get_material_matches(bounds_dict)
print(f'Material Matches: {matches_dict}')

Retrieving SummaryDoc documents:   0%|          | 0/261 [00:00<?, ?it/s]

Material Matches: {'mat1': [{'mp-684591': {'elec_cond': 87.8933, 'therm_cond': 0.000354247, 'bulk_modulus': 141.633, 'shear_modulus': 83.335, 'universal_anisotropy': 1.2690000000000001}}, {'mp-752826': {'elec_cond': 41.0475, 'therm_cond': 0.000111994, 'bulk_modulus': 166.16, 'shear_modulus': 81.144, 'universal_anisotropy': 1.271}}, {'mp-3098': {'elec_cond': 80.5092, 'therm_cond': 0.00032156, 'bulk_modulus': 178.355, 'shear_modulus': 80.28, 'universal_anisotropy': 2.207}}, {'mp-3748': {'elec_cond': 115.666, 'therm_cond': 0.000491413, 'bulk_modulus': 156.0, 'shear_modulus': 81.731, 'universal_anisotropy': 2.097}}, {'mp-3536': {'elec_cond': 77.0117, 'therm_cond': 0.000316911, 'bulk_modulus': 182.861, 'shear_modulus': 98.235, 'universal_anisotropy': 1.062}}, {'mp-4391': {'elec_cond': 110.485, 'therm_cond': 0.000659704, 'bulk_modulus': 199.288, 'shear_modulus': 108.878, 'universal_anisotropy': 1.601}}, {'mp-5924': {'elec_cond': 64.5658, 'therm_cond': 0.000300167, 'bulk_modulus': 189.894, 's

If the above cell fails, the Materials Project API may have been updated. Run the following cell and then re-run the entire notebook.

In [ ]:
!pip install --upgrade mp-api